# Training Pipeline

## imports

In [1]:
!pip install tensorflow --quiet

In [2]:
import inspect 
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

2024-04-15 13:15:24.726608: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Connecting to Hopsworks Feature Store

In [3]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/550040
Connected. Call `.close()` to terminate connection gracefully.


In [5]:
# Retrieve feature groups
electricity_fg = fs.get_feature_group(
    name='electricity_prices',
    version=1,
)

weather_fg = fs.get_feature_group(
    name='weather_measurements',
    version=1,
)

danish_holidays_fg = fs.get_feature_group(
    name='danish_holidays',
    version=1,
)

In [8]:
# Select features for training data
selected_features = electricity_fg.select_all()\
    .join(
    weather_fg\
        .select_except(["timestamp"])
    )\
    .join(
        danish_holidays_fg.select_all()
    )

In [19]:
transformation_functions = {
        "SpotPriceDKK_KWH": fs.get_transformation_function(name="min_max_scaler"), 
        "temperature_2m": fs.get_transformation_function(name="min_max_scaler"), 
        "relative_humidity_2m": fs.get_transformation_function(name="min_max_scaler"), 
        "precipitation": fs.get_transformation_function(name="min_max_scaler"), 
        "rain": fs.get_transformation_function(name="min_max_scaler"), 
        "snowfall": fs.get_transformation_function(name="min_max_scaler"), 
        "weather_code": fs.get_transformation_function(name="min_max_scaler"), 
        "cloud_cover": fs.get_transformation_function(name="min_max_scaler"), 
        "wind_speed_10m": fs.get_transformation_function(name="min_max_scaler"),
        "wind_gusts_10m": fs.get_transformation_function(name="min_max_scaler")
    }

In [20]:
feature_view = fs.get_or_create_feature_view(
    name='electricity_feature_view',
    version=1,
    labels=[], # you will define our 'y' later manualy
    transformation_functions=transformation_functions,
    query=selected_features,
)

FeatureStoreException: Feature name 'SpotPriceDKK_KWH' could not found be found in query.